In [1]:
using CSV, DataFrames, GLM, RegressionTables, c
using Statistics

[ Info: Precompiling CSV [336ed68f-0bac-5ca0-87d4-7b16caf5d00b] (cache misses: incompatible header (2))
[ Info: Precompiling DataFrames [a93c6f00-e57d-5684-b7b6-d8193f3e46c0] (cache misses: incompatible header (2))
[ Info: Precompiling GLM [38e38edf-8417-5370-95a0-9cbb8c7f171a] (cache misses: incompatible header (2))
[ Info: Waiting for another process (pid: 57502) to finish precompiling RegressionTables [d519eb52-b820-54da-95a6-98e1306fdade]. Pidfile: /home/gergo/.julia/compiled/v1.11/RegressionTables/cYvie_aAiQW.ji.pidfile
[ Info: Precompiling RegressionTablesGLMExt [6eb700d4-e7d6-5b1a-bdb7-659d42738381] (cache misses: incompatible header (2))


LoadError: ArgumentError: Package c not found in current path.
- Run `import Pkg; Pkg.add("c")` to install the c package.

In [76]:
function scale(df::DataFrame; cols::Vector{String}=names(df))
    result = copy(df)
    for col in cols
        result[!, col] = (df[!, col] .- mean(df[!, col])) ./ std(df[!, col])
    end
    return result
end

function scale(x::AbstractVector)
    return (x .- mean(x)) ./ std(x)
end

scale (generic function with 2 methods)

In [2]:
CITY = "helsinki"
VERSION = "20250428"
YEAR = 2022
;

In [96]:
data = CSV.read("../output/$CITY/$VERSION/merged.csv", DataFrame)
socioecon = CSV.read("../output/$CITY//socioecon_change.csv", DataFrame)
;
data = innerjoin(data, socioecon, on = :stop_id)
;
# data.total_multimodal = scale(data.total_multimodal)
# data.total_walk15 = scale(data.total_walk15)
# data.area_difference = scale(data.area_difference)
# data.distance_betweenness = scale(data.distance_betweenness)
;

In [88]:
# select(data, Not(:geometry))

In [69]:
# data = scale(data, cols=setdiff(names(data), ["stop_id", "geometry", "stop_name", "stop_lat", "stop_lon", "ellipticity"]))
;

In [ ]:
# data[!, :gini_change] = data[!, :multimodal_gini] .- data[!, :walk15_gini]
;

In [103]:
# helsinki1
# helsinki_gini = reg(data, @formula(weighted_gini_multi - weighted_gini_walk ~ weighted_gini_walk + (area_difference * ellipticity) + (distance_betweenness * ellipticity)))
helsinki_gini = GLM.lm(@formula(weighted_gini_multi - weighted_gini_walk ~ weighted_gini_walk + (area_difference * ellipticity) + (distance_betweenness * ellipticity)), data)
# helsinki1a
# helsinki_access = reg(data, @formula(total_multimodal - total_walk15 ~ total_walk15 + (area_difference*ellipticity) + (distance_betweenness*ellipticity)))
helsinki_access = GLM.lm(@formula(total_multimodal - total_walk15 ~ total_walk15 + (area_difference*ellipticity) + (distance_betweenness*ellipticity)), data)
;

In [106]:
println(coeftable(helsinki_access))

────────────────────────────────────────────────────────────────────────────────────────────────────
                                         Coef.  Std. Error      t  Pr(>|t|)    Lower 95%   Upper 95%
────────────────────────────────────────────────────────────────────────────────────────────────────
(Intercept)                         -75.3995    38.8948     -1.94    0.0527  -151.666       0.867252
total_walk15                          0.652123   0.0164174  39.72    <1e-99     0.619931    0.684315
area_difference                      38.766      4.79293     8.09    <1e-15    29.3678     48.1642
ellipticity                         335.461     65.6205      5.11    <1e-06   206.789     464.132
distance_betweenness                 -5.79035    4.90384    -1.18    0.2378   -15.406       3.82533
area_difference & ellipticity        31.3209     8.25055     3.80    0.0002    15.1428     47.499
distance_betweenness & ellipticity  -28.5244     8.20917    -3.47    0.0005   -44.6213    -12.4275
─

In [104]:
regtable([helsinki_gini, helsinki_access]...; render=AsciiTable(), labels=Dict("weighted_gini_multi - weighted_gini_walk" => "Helsinki - Gini", "total_multimodal - total_walk15" => "Helsinki - Access"))


------------------------------------------------------------------------
                                     Helsinki - Gini   Helsinki - Access
                                     ---------------   -----------------
                                                 (1)                 (2)
------------------------------------------------------------------------
(Intercept)                                 0.143***             -75.400
                                             (0.003)            (38.895)
weighted_gini_walk                         -0.425***                    
                                             (0.010)                    
area_difference                             -0.001**           38.766***
                                             (0.000)             (4.793)
ellipticity                                -0.015***          335.461***
                                             (0.003)            (65.620)
distance_betweenness                       -0.001*

In [ ]:
# m0 = reg(data, @formula(gini_change ~ area + distance_betweenness + closeness_centrality + ellipticity))
m2 = reg(data, @formula(multimodal_gini ~ walk15_gini + area + distance_betweenness + closeness_centrality + ellipticity))

                                        FixedEffectModel                                        
Number of obs:                             4425  Converged:                                 true
dof (model):                                  5  dof (residuals):                           4418
R²:                                       0.204  R² adjusted:                              0.203
F-statistic:                            226.782  P-value:                                  0.000
                          Estimate   Std. Error     t-stat  Pr(>|t|)     Lower 95%     Upper 95%
────────────────────────────────────────────────────────────────────────────────────────────────
walk15_gini            0.286285     0.0090286    31.7086      <1e-99   0.268584      0.303985
area                  -5.56795e-5   0.000143451  -0.388143    0.6979  -0.000336915   0.000225556
distance_betweenness   0.000497824  0.000102409   4.86114     <1e-05   0.000297051   0.000698596
closeness_centrality   0.385421  

In [ ]:
regtable([m2]...; render=AsciiTable(), file="m1.txt")


--------------------------------------
                       multimodal_gini
--------------------------------------
(Intercept)                   0.099***
                               (0.003)
walk15_gini                   0.286***
                               (0.009)
area                            -0.000
                               (0.000)
distance_betweenness          0.000***
                               (0.000)
closeness_centrality          0.385***
                               (0.032)
ellipticity                   -0.004**
                               (0.001)
--------------------------------------
N                                4,425
R2                               0.204
--------------------------------------

